In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt

from Training import GetDatasets
from Nets  import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from Simulation import buffer_simulation, save_obj, load_obj


tf.random.set_seed(1234)
np.random.seed(1234)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


a) Get Dataset

In [5]:
train_batch_size = test_batch_size = 32

train_set,valid_set,test_set = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, train_batch_size, test_batch_size)

a) Get Model

In [4]:

tfds.load('colorectal_histology')

NameError: name 'tfds' is not defined

Se crea la red, sin activar la cuantización ni el efecto de envejecimiento
Las dimensiones de entrada de la imagen (227,227), el número de clases (8) y el tamaño de los batches

In [6]:
ZFNet   = GetNeuralNetworkModel('ZFNet',(224,224,3),8, quantization = False, aging_active=False)
loss      = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
ZFNet.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [7]:
for index,layer in enumerate(ZFNet.layers):
    print(index,layer.name)

0 input_1
1 lambda
2 lambda_1
3 conv2d
4 re_lu
5 lambda_2
6 lambda_3
7 max_pooling2d
8 lambda_4
9 lambda_5
10 lambda_6
11 conv2d_1
12 re_lu_1
13 lambda_7
14 lambda_8
15 max_pooling2d_1
16 lambda_9
17 lambda_10
18 lambda_11
19 conv2d_2
20 re_lu_2
21 lambda_12
22 lambda_13
23 conv2d_3
24 re_lu_3
25 lambda_14
26 lambda_15
27 conv2d_4
28 re_lu_4
29 lambda_16
30 lambda_17
31 max_pooling2d_2
32 lambda_18
33 flatten
34 dense
35 lambda_19
36 lambda_20
37 dense_1
38 lambda_21
39 lambda_22
40 dense_2
41 lambda_23
42 activation
43 lambda_24


 b) Load/Save Weigths

In [15]:
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'ZFNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir, 'Weights') 
ZFNet.load_weights(wgt_dir)

ValueError: Shapes (11, 11, 3, 96) and (7, 7, 3, 96) are incompatible

Evaluación del accuracy y loss de la red

In [7]:
(OrigLoss,OrigAcc) = ZFNet.evaluate(test_set)
print(test_set)

24/24 [==============================] - 75s 3s/step - loss: 0.2809 - accuracy: 0.9133
<BatchDataset shapes: ((None, 227, 227, 3), (None, 8)), types: (tf.float32, tf.float32)>


2) Stats

 Write/Read Stats¶

Se identifican (manualmente) las capas procesadadas(Convoluciones, Full conectadas y Pooling) junto a las capas que contienen los resultados que se escribiran en el buffer (capas luego de la funcion de activacion y/o Normalizacion)

In [8]:
for index,layer in enumerate(ZFNet.layers):
    print(index,layer.name)
print('Las capas 0,3,9,11,17,19,25,31,37,40,45 y 50  contienen la informacion para su procesamiento en MMU')
print('Las capas 2,8,10,16,18,24,30,36,38,44,49 y 53 contienen las activaciones que son escritas en memoria')

0 input_1
1 lambda
2 lambda_1
3 zero_padding2d
4 conv2d
5 lambda_2
6 batch_normalization
7 re_lu
8 lambda_3
9 lambda_4
10 zero_padding2d_1
11 max_pooling2d
12 batch_normalization_1
13 re_lu_1
14 lambda_5
15 lambda_6
16 conv2d_1
17 lambda_7
18 batch_normalization_2
19 re_lu_2
20 lambda_8
21 lambda_9
22 conv2d_2
23 lambda_10
24 concatenate
25 batch_normalization_3
26 re_lu_3
27 lambda_11
28 lambda_12
29 conv2d_3
30 lambda_13
31 batch_normalization_4
32 re_lu_4
33 lambda_14
34 lambda_15
35 conv2d_4
36 lambda_16
37 concatenate_1
38 batch_normalization_5
39 re_lu_5
40 lambda_17
41 lambda_18
42 conv2d_5
43 lambda_19
44 batch_normalization_6
45 re_lu_6
46 lambda_20
47 lambda_21
48 conv2d_6
49 lambda_22
50 concatenate_2
51 batch_normalization_7
52 re_lu_7
53 lambda_23
54 lambda_24
55 conv2d_7
56 lambda_25
57 batch_normalization_8
58 re_lu_8
59 lambda_26
60 lambda_27
61 conv2d_8
62 lambda_28
63 concatenate_3
64 batch_normalization_9
65 re_lu_9
66 lambda_29
67 lambda_30
68 conv2d_9
69 lambda_31


Con el siguiente bloque obtenemos el número de lecturas y escrituras por posición de memoria tanto usando la estrategia de CNN Gating o sin usarla

In [9]:
Indices = [0,3,7,11,15,19,23,27,31,34,37,40]
Data    = GetReadAndWrites(ZFNet,Indices,279936,150,CNN_gating=False)
stats   = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
Data    = GetReadAndWrites(ZFNet,Indices,1024*1024,150,CNN_gating=True)
stats   = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
CNN_gating_Acceses = pd.DataFrame(stats).reset_index(drop=False)
save_obj(Baseline_Acceses,'Data/Acceses/ZFNet/Baseline_v2')
save_obj(CNN_gating_Acceses,'Data/Acceses/ZFNet/CNN_gating_Fix_v2')

AttributeError: 'ZeroPadding2D' object has no attribute 'kernel_size'

Analizar la posibilidad de usar menos bits

In [12]:
CheckAccuracyAndLoss('DenseNet', test_set, wgt_dir, act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4, 
                    input_shape = (227,227,3), output_shape = 8, batch_size = test_batch_size);

24/24 [==============================] - 142s 6s/step - loss: 0.2976 - accuracy: 0.9120


 c) Activation Stats

Para la configuración anterior, se observa el valor medio,maximo,minimo y el ratio de saturación tanto de las activaciones procesadas dentro de la unidad matricial de multiplicacion como de las almacenadas en el buffer. Nota: el ultimo parametro indica el numero de iteraciones que se deben realizar hasta agotar el dataset, se obtiene como numero de imagenes del dataset dividido en el batch size.

In [ ]:
ActivationStats(DenseNet,test_set,11,4,24)

3) Buffer Simulation

 a) Baseline

Ahora para el Baseline simularemos el comportamiento de 1 buffer durante la inferencia de 3 imagenes (solo 3 como ejemplo), la red se crea ahora activando la cuantizacion pero no el envejecimiento. LI y AI son los definidos en el item 2) Stats

......estos ficheros ya están , solo cargar e interpretarlos

Sin inyectar errores original

In [ ]:
# from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import os

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'DenseNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


Accs_x= []



network_size   = 290400*16   # Tamaño del buffer (en bits)
num_of_samples = 1       # Numero de muestras de distintas configuraciones de fallos a testear por cada valor de Accs/Loss
    
n_bits_total = np.ceil(network_size).astype(int)    #numero de bits totales
    
buffer = np.array(['x']*(network_size))
print(buffer)
for index in range(0,num_of_samples):
        
    loss,acc_x   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = False)
       
        
    print(acc_x)
        
      
        
    Accs_x.append(acc_x)
        
print(str(n_bits_total)+' completada: ', datetime.now().strftime("%H:%M:%S"))
#save_obj(Accs_x,'Data/Errors/DenseNet/Colorectal Dataset/Accs_x')
#save_obj(Loss,'Data/Errors/DenseNet/Colorectal Dataset/Loss')

# Analizando el fichero en su estado original

In [ ]:
## from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

 
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'DenseNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216

       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)
num_of_samples = 1

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
    
        loss,acc   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_w   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_a_w   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    
        Accs.append(acc)
        Accs_w.append(acc_w)
        Accs_a_w.append(acc_a_w)                                      


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_w)                                         
#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)


Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#lsave_file(Accs,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/DenseNet/Colorectal Dataset/Loss')

In [1]:
16777216/16

1048576.0

# Inyectando errores en 1

In [ ]:
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)




    

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'DenseNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

Accs_1 = []
Accs_w_1 = []
Accs_a_w_1 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
print(directorio)
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1.append(acc_1)
        Accs_w_1.append(acc_w_1)
        Accs_a_w_1.append(acc_a_w_1)                                      


Acc_1=pd.DataFrame(Accs_1)
Acc_w_1 =pd.DataFrame(Accs_w_1)
Acc_a_w_1 =pd.DataFrame(Accs_w_1)                                         
#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)



Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/DenseNet/Colorectal Dataset/Loss')

# Inyectando errores aleatorios ceros y unos

In [ ]:
buffer_size= 16777216
Accs_1_0 = []
Accs_w_1_0 = []
Accs_a_w_1_0 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]

for index in range(0,num_of_samples):

    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno_ceros(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1_0   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1_0   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1_0   = CheckAccuracyAndLoss('DenseNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1_0.append(acc_1_0)
        Accs_w_1_0.append(acc_w_1_0)
        Accs_a_w_1_0.append(acc_a_w_1_0)                                      


Acc_1_0=pd.DataFrame(Accs_1_0)
Acc_w_1_0 =pd.DataFrame(Accs_w_1_0)
Acc_a_w_1_0 =pd.DataFrame(Accs_w_1_0)                                         
buf_cero = pd.concat([Acc,Acc_w, Acc_a_w,Acc_1,Acc_w_1, Acc_a_w_1,Acc_1_0,Acc_w_1_0, Acc_a_w_1_0], axis=1, join='outer')
buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero', 'Acc_uno', 'A_w_uno', 'Acc_a_w_uno' ,'Acc_uno_cero', 'A_w_uno_cero', 'Acc_a_w_uno_cero'] 
buf_cero.to_excel('resultado.xlsx', sheet_name='fichero_707', index=False)

#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)
Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/DenseNet/Colorectal Dataset/Loss')

# Gráficas